In [2]:
import pandas as pd
import numpy as np
import re
from IPython.display import display

## FT CS DATA PREP

2

#### import and merge data

In [3]:
# import nzz cs data

file_paths = [
    "Data/FT/FT_data_CS_batch1.csv",
    "Data/FT/FT_data_CS_batch2.csv",
    "Data/FT/FT_data_CS_batch3.csv",
    "Data/FT/FT_data_CS_batch4.csv"
]

# List to store DataFrames
dfs = []

# Read each CSV file into a DataFrame and append to the list
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
ft_data_CS_merged = pd.concat(dfs, ignore_index=True)

In [4]:
len(ft_data_CS_merged)

3051

In [5]:
ft_data_CS_merged.head()

,article_title,headline,article_content,text_date
0,UBS chief Ermotti hints at keeping Credit Suis...,Shotgun marriage between lenders has attracted...,UBS chief executive Sergio Ermotti has given t...,17.06.2023
1,Credit Suisse’s lawsuit against SoftBank throw...,Japanese conglomerate is client of Swiss bank ...,Credit Suisse’s London High Court lawsuit agai...,21.06.2023
2,UBS sets ‘red lines’ for Credit Suisse staff a...,Diktat is designed to reduce risks of the tran...,UBS has imposed tight restrictions on Credit S...,12.06.2023
3,UBS considers delaying results after Credit Su...,Pushing second-quarter results until the end o...,UBS is considering delaying its second-quarter...,04.06.2023
4,UBS prioritises Credit Suisse investment bank ...,Decision on Swiss domestic unit expected withi...,UBS has prioritised integrating Credit Suisse’...,12.05.2023


#### remove duplicates

In [6]:
# Check for duplicates in the merged DataFrame, keeping all occurrences
all_occurrences_indices = ft_data_CS_merged[ft_data_CS_merged.duplicated(keep=False)].index

# Display the indices of all occurrences of duplicate rows
print("Indices of All Occurrences of Duplicate Rows:")
print(all_occurrences_indices)

Indices of All Occurrences of Duplicate Rows:
Index([], dtype='int64')


In [7]:
duplicate_indices_specific_columns = ft_data_CS_merged[ft_data_CS_merged.duplicated(subset=['article_content'], keep=False)].index
print("Indices of Duplicate Rows Based on Specific Columns:")
print(duplicate_indices_specific_columns)

Indices of Duplicate Rows Based on Specific Columns:
Index([], dtype='int64')


In [100]:
# Remove duplicate rows based on specific columns --> No duplicates found
#ft_data_CS_merged_no_dups = ft_data_CS_merged.drop_duplicates(subset=['article_content'], keep='first')

In [39]:
#nzz_data_CS_merged.loc[126, "article_content"]

#### Filter by date

In [8]:
# Create a copy of the DataFrames
ft_data_CS_merged_copy = ft_data_CS_merged.copy()

# Convert 'text_date' column to datetime type, considering timezone information
ft_data_CS_merged_copy['text_date'] = pd.to_datetime(ft_data_CS_merged_copy['text_date'], utc=True, format='%d.%m.%Y', errors='coerce')

# Define the start and end dates of the desired range
start_date = pd.Timestamp('2019-01-01', tz='UTC')
end_date = pd.Timestamp('2023-06-30', tz='UTC')

# Filter the DataFrame to include only articles within the specified date range
filtered_df_ft_cs = ft_data_CS_merged_copy[(ft_data_CS_merged_copy['text_date'] >= start_date) & (ft_data_CS_merged_copy['text_date'] <= end_date)]

#### check if other variants of Credit Suisse are in the articles.

In [28]:
#filtered_df_ft_cs_copy = filtered_df_ft_cs.copy()

# Create a boolean Series where each element is True if the corresponding article contains 'credit suisse' or 'CREDIT SUISSE'

#contains_lower = filtered_df_ft_cs_copy['article_content'].str.contains('Credit Suisse', case=True)
#contains_upper = filtered_df_ft_cs_copy['article_content'].str.contains('Credit_Suisse', case=True)

# Combine the two Series using the bitwise OR operator

#contains_either = contains_lower | contains_upper

# Check if any articles contain 'credit suisse' or 'CREDIT SUISSE'

#any_contains_either = contains_either.any()

#print(f"Any articles contain 'credit suisse' or 'CREDIT SUISSE'? {any_contains_either}")

#num_contains_either = contains_either.sum()

#print(f"Number of articles that contain 'credit suisse' or 'CREDIT SUISSE': {num_contains_either}")

#### Filter by relevancy

##### old code to filter by relevancy

In [51]:
# Create a copy of the filtered DataFrame
#filtered_df_ft_cs_copy = filtered_df_ft_cs.copy()

# Count the occurrences of the phrases 'Credit Suisse' and 'CS' in each article's content in the copied DataFrame
#filtered_df_ft_cs_copy['credit_suisse_count'] = filtered_df_ft_cs_copy['article_content'].str.count('Credit Suisse')

#def count_cs(text):
#    return len(re.findall(r'CS(?![a-zA-Z])', text))

#filtered_df_ft_cs_copy['cs_count'] = filtered_df_ft_cs_copy['article_content'].apply(count_cs)

# Filter articles based on multiple conditions
#ft_credit_suisse_articles_pre = filtered_df_ft_cs_copy[
#    (filtered_df_ft_cs_copy['credit_suisse_count'] >= 2) |
#    (filtered_df_ft_cs_copy['cs_count'] >= 2) |
#    ((filtered_df_ft_cs_copy['credit_suisse_count'] >= 1) & (filtered_df_ft_cs_copy['cs_count'] >= 1))
#]


In [9]:
print(len(filtered_df_ft_cs))
len(ft_credit_suisse_articles_pre)

3051


NameError: name 'ft_credit_suisse_articles_pre' is not defined

##### new code to filter by relevancy

In [10]:
# Create a copy of the filtered DataFrame
filtered_df_ft_cs_copy = filtered_df_ft_cs.copy()

# Count the occurrences of the phrases 'Credit Suisse' and 'CS' in each article's content in the copied DataFrame
def count_credit_suisse(text):
    return len(re.findall(r'credit\s*suisse', text, re.IGNORECASE))
    
filtered_df_ft_cs_copy['credit_suisse_count'] = filtered_df_ft_cs_copy['article_content'].apply(count_credit_suisse)

def count_cs(text):
    return len(re.findall(r'(?<![a-zA-Z])CS(?![a-zA-Z])', text))

filtered_df_ft_cs_copy['cs_count'] = filtered_df_ft_cs_copy['article_content'].apply(count_cs)

# Filter articles based on multiple conditions
ft_credit_suisse_articles = filtered_df_ft_cs_copy[
    (filtered_df_ft_cs_copy['credit_suisse_count'] >= 2) |
    (filtered_df_ft_cs_copy['cs_count'] >= 2) |
    ((filtered_df_ft_cs_copy['credit_suisse_count'] >= 1) & (filtered_df_ft_cs_copy['cs_count'] >= 1))
]

In [11]:
print(len(filtered_df_ft_cs))
len(ft_credit_suisse_articles)

3051


1291

##### get the differences

In [12]:
# Get the titles of the articles in ft_credit_suisse_articles_pre that are not in ft_credit_suisse_articles_after
missing_titles = ft_credit_suisse_articles_pre[~ft_credit_suisse_articles_pre['article_title'].isin(ft_credit_suisse_articles_after['article_title'])]['article_title']

print(missing_titles)

NameError: name 'ft_credit_suisse_articles_pre' is not defined

In [15]:
# Export the unfiltered DataFrame to a CSV file
#filtered_df_ft_cs_copy.to_csv('ft_cs_unfiltered.csv', index=False)

#### Calculate word count

In [14]:
# Calculate the word count for each article in the credit_suisse_articles DataFrame
ft_credit_suisse_articles.loc[:, 'word_count'] = ft_credit_suisse_articles['article_content'].str.split().apply(len)

#### Exclude certain types of articles

In [15]:
# List of prefixes to exclude
exclude_prefixes = [
    "Episode",
    "FirstFT",
    "Markets not live",
    "Markets Now",
    "Opening Quote:",
    "Stocks to watch:"
]

# Filter the DataFrame to exclude articles with titles starting with specified prefixes
filtered_prefixes_df = ft_credit_suisse_articles[~ft_credit_suisse_articles['article_title'].str.startswith(tuple(exclude_prefixes))]


In [16]:
len(filtered_prefixes_df)

1156

#### remove over 5000 k words, sort and export

In [17]:
# Sort the articles by date in ascending order
ft_credit_suisse_articles_sorted = filtered_prefixes_df.sort_values(by='text_date')

In [18]:
# remove over 5k words
# Filter out articles with a word count higher than 5000
ft_credit_suisse_articles_sorted = ft_credit_suisse_articles_sorted[ft_credit_suisse_articles_sorted['word_count'] <= 5000]

In [19]:
print(len(ft_credit_suisse_articles_sorted))
len(ft_credit_suisse_articles)

1155


1291

In [69]:
# change column name of article content to article_content if necessary (einheitlich machen across alle dfs) 
ft_credit_suisse_articles_sorted.columns


Index(['article_title', 'headline', 'article_content', 'text_date',
       'credit_suisse_count', 'cs_count', 'word_count'],
      dtype='object')

In [70]:
# Export the sorted DataFrame to a CSV file
ft_credit_suisse_articles_sorted.to_csv('Prepped_Data/CS_FT_prepped.csv', index=False)

In [71]:
ft_credit_suisse_articles_sorted.head()

,article_title,headline,article_content,text_date,credit_suisse_count,cs_count,word_count
2691,Credit Suisse escapes harsh criticism on ‘tuna...,US prosecutors seem to take sympathetic view o...,The arrest of three former Credit Suisse banke...,2019-01-04 00:00:00+00:00,11,0,696
2677,Ex-Credit Suisse bankers charged over $2bn Moz...,Country’s one-time finance minister also arres...,Three former Credit Suisse bankers have been c...,2019-01-04 00:00:00+00:00,7,0,719
2635,Tuna bond scandal may spell more trouble for C...,Prosecutors could yet conclude that the bank p...,© Bloomberg When the US Department of Justice ...,2019-01-08 00:00:00+00:00,9,1,631
2917,"Swiss plans for whistleblowers fall short, cri...",Staff who report wrongdoing risk losing their ...,Swiss government plans to update laws on whist...,2019-01-08 00:00:00+00:00,2,0,660
2928,Opinion today: Why the world economy feels so ...,Mild slowdown is to be expected given how long...,This article is from today’s FT Opinion email....,2019-01-09 00:00:00+00:00,2,0,872


## ft ubs data

#### import and merge data

In [91]:
# import nzz cs data

file_paths = [
    "Data/FT/FT_data_UBS_batch1.csv",
    "Data/FT/FT_data_UBS_batch2.csv",
    "Data/FT/FT_data_UBS_batch3.csv",
    "Data/FT/FT_data_UBS_batch4.csv"
]

# List to store DataFrames
dfs = []

# Read each CSV file into a DataFrame and append to the list
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
ft_data_UBS_merged = pd.concat(dfs, ignore_index=True)

In [92]:
len(ft_data_UBS_merged)

3159

In [93]:
ft_data_UBS_merged.head()

,article_title,headline,article_content,text_date
0,UBS chief Ermotti hints at keeping Credit Suis...,Shotgun marriage between lenders has attracted...,UBS chief executive Sergio Ermotti has given t...,17.06.2023
1,UBS sets ‘red lines’ for Credit Suisse staff a...,Diktat is designed to reduce risks of the tran...,UBS has imposed tight restrictions on Credit S...,12.06.2023
2,UBS considers delaying results after Credit Su...,Pushing second-quarter results until the end o...,UBS is considering delaying its second-quarter...,04.06.2023
3,UBS prioritises Credit Suisse investment bank ...,Decision on Swiss domestic unit expected withi...,UBS has prioritised integrating Credit Suisse’...,12.05.2023
4,Credit Suisse’s lawsuit against SoftBank throw...,Japanese conglomerate is client of Swiss bank ...,Credit Suisse’s London High Court lawsuit agai...,21.06.2023


#### remove duplicates

In [94]:
# Check for duplicates in the merged DataFrame, keeping all occurrences
all_occurrences_indices = ft_data_UBS_merged[ft_data_UBS_merged.duplicated(keep=False)].index

# Display the indices of all occurrences of duplicate rows
print("Indices of All Occurrences of Duplicate Rows:")
print(all_occurrences_indices)

Indices of All Occurrences of Duplicate Rows:
Index([], dtype='int64')


In [95]:
duplicate_indices_specific_columns = ft_data_UBS_merged[ft_data_UBS_merged.duplicated(subset=['article_content'], keep=False)].index
print("Indices of Duplicate Rows Based on Specific Columns:")
print(duplicate_indices_specific_columns)

Indices of Duplicate Rows Based on Specific Columns:
Index([], dtype='int64')


In [70]:
# Remove duplicate rows based on specific columns
# --> no duplicates
#ft_data_UBS_merged_no_dups = ft_data_UBS_merged.drop_duplicates(subset=['article_content'], keep='first')

In [81]:
#ft_data_UBS_merged.loc[2360, "article_content"]

#### filter date

In [96]:
# Create a copy of the DataFrames
ft_data_UBS_merged_copy = ft_data_UBS_merged.copy()

# Convert 'text_date' column to datetime type, considering timezone information
ft_data_UBS_merged_copy['text_date'] = pd.to_datetime(ft_data_UBS_merged_copy['text_date'], utc=True, format='%d.%m.%Y', errors='coerce')

# Define the start and end dates of the desired range
start_date = pd.Timestamp('2019-01-01', tz='UTC')
end_date = pd.Timestamp('2023-06-30', tz='UTC')

# Filter the DataFrame to include only articles within the specified date range
filtered_df_ft_ubs = ft_data_UBS_merged_copy[(ft_data_UBS_merged_copy['text_date'] >= start_date) & (ft_data_UBS_merged_copy['text_date'] <= end_date)]

In [31]:
len(filtered_df_ft_ubs)

3159

#### filter by relevancy

##### old code 

In [83]:
# Create a copy of the filtered DataFrame
filtered_df_ft_ubs_copy = filtered_df_ft_ubs.copy()

# Count the occurrences of the phrases 'Credit Suisse' and 'CS' in each article's content in the copied DataFrame
filtered_df_ft_ubs_copy['UBS_count'] = filtered_df_ft_ubs_copy['article_content'].str.count('UBS')

# Filter articles containing 'Credit Suisse' at least twice in their content
ft_UBS_articles_pre = filtered_df_ft_ubs_copy[filtered_df_ft_ubs_copy['UBS_count'] >= 2]

In [85]:
print(len(filtered_df_ft_ubs))
print(len(ft_UBS_articles_pre))

3159
1049


##### new code

In [97]:
# Create a copy of the filtered DataFrame
filtered_df_ft_ubs_copy = filtered_df_ft_ubs.copy()

# Count the occurrences of the phrases "UBS" in each article's content in the copied DataFrame
def count_ubs(text):
    return len(re.findall(r'(?<![a-zA-Z])UBS(?![a-zA-Z])', text))

filtered_df_ft_ubs_copy['UBS_count'] = filtered_df_ft_ubs_copy['article_content'].apply(count_ubs)

# Filter articles containing 'UBS' at least twice in their content
ft_UBS_articles = filtered_df_ft_ubs_copy[filtered_df_ft_ubs_copy['UBS_count'] >= 2]

In [98]:
print(len(filtered_df_ft_ubs))
print(len(ft_UBS_articles))

3159
1036


In [91]:
# Export the sorted DataFrame to a CSV file
#filtered_df_nzz_ubs_copy.to_csv('UBS_NZZ_unfiltered.csv', index=False)

In [88]:
# Get the titles of the articles in ft_UBS_articles_pre that are not in ft_UBS_articles
missing_titles_ubs = ft_UBS_articles_pre[~ft_UBS_articles_pre['article_title'].isin(ft_UBS_articles['article_title'])]['article_title']

print(missing_titles_ubs)

343                              Peace on Earth postponed
832     A new chapter for space travel and an annivers...
1635                   Can corporate Europe catch the US?
2099     Credit Suisse freezes investment bank bonus pool
2174                 Markets Now - Thursday 3rd September
2212             Markets Now - Wednesday 12th August 2020
2217               Markets Now - Tuesday 18th August 2020
2224                 Markets Now - Wednesday 6th May 2020
2230           Markets not live, Monday 13th January 2020
2237        Markets not live, Wednesday 5th February 2020
2240                Markets now, Wednesday 4th March 2020
2248          Markets Now - LIVE - Friday 13th March 2020
2250                Markets Now - Tuesday 17th March 2020
Name: article_title, dtype: object


#### calculate wordcount

In [100]:
# Calculate the word count for each article in the credit_suisse_articles DataFrame
ft_UBS_articles.loc[:, 'word_count'] = ft_UBS_articles['article_content'].str.split().apply(len)

##### exclude certain titles

In [101]:
# List of prefixes to exclude
exclude_prefixes = [
    "Episode",
    "FirstFT",
    "Markets not live",
    "Markets Now",
    "Opening Quote:",
    "Stocks to watch:"
]

# Filter the DataFrame to exclude articles with titles starting with specified prefixes
filtered_prefixes_df2 = ft_UBS_articles[~ft_UBS_articles['article_title'].str.startswith(tuple(exclude_prefixes))]

#### remove over 5k, sort and export

In [102]:
 #Sort the articles by date in ascending order
ft_UBS_articles_sorted = filtered_prefixes_df2.sort_values(by='text_date')

In [103]:
len(ft_UBS_articles_sorted)

939

In [104]:
# remove over 5k words
# Filter out articles with a word count higher than 5000
ft_UBS_articles_sorted = ft_UBS_articles_sorted[ft_UBS_articles_sorted['word_count'] <= 5000]

In [105]:
print(len(ft_UBS_articles_sorted))
len(ft_UBS_articles)

939


1036

In [106]:
# change column name of article content to article_content if necessary (einheitlich machen across alle dfs) 
ft_UBS_articles_sorted.columns


Index(['article_title', 'headline', 'article_content', 'text_date',
       'UBS_count', 'word_count'],
      dtype='object')

In [107]:
# Export the sorted DataFrame to a CSV file
ft_UBS_articles_sorted.to_csv('Prepped_Data/UBS_FT_prepped.csv', index=False)

In [108]:
ft_UBS_articles_sorted.head()

,article_title,headline,article_content,text_date,UBS_count,word_count
2868,Alphabet overtakes Apple as world’s third-bigg...,The iPhone maker saw its shares slide 9% after...,Apple is falling further down the rankings of ...,2019-01-03 00:00:00+00:00,2,230
2547,UBS in talks to hire ex-BofA investment bankin...,Christian Meissner has had talks about joining...,UBS has sounded out former Bank of America inv...,2019-01-04 00:00:00+00:00,9,448
2605,The London Report: Poor spell for Merlin Enter...,UBS sounds note of caution from customer feedb...,Merlin Entertainments\r\n was among the day’s ...,2019-01-10 00:00:00+00:00,2,162
3084,Malaysian electronics firms face hit from China,Sector braced for slowdown as over-reliance on...,Malaysia’s electronics sector is bracing for a...,2019-01-10 00:00:00+00:00,2,850
3079,Probe builds pressure on insurers to end ‘loya...,Industry set for shake-up as regulator’s inves...,The way insurers decide what to charge loyal c...,2019-01-10 00:00:00+00:00,2,872
